# Verify Model with MinMax
TODO: Cluster with KMeans

In [809]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [810]:
schemas_path = "../data/schemas.txt"
# Create the dataframe
df = pd.read_csv(schemas_path, sep=";")
df.head()

,Type,Modulus,Size,Skip Values,Skip Type,Result


In [811]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [812]:
def get_mean(data):
    """
    Take a string of numbers seperated by commas.
    Split the string by the commas.
    Find the mean of the numbers.
    """
    numbers = list(map(int, data.split(",")))
    return np.mean(numbers)

In [813]:
def get_average(data):
    """
    Take a string of numbers seperated by commas.
    Splt the string by the commas.
    Find the average of the numbers
    """
    numbers = list(map(int, data.split(",")))
    return np.average(numbers)

In [814]:
def get_type(data):
    """
    Return the Schema type as a number.
    Circles => 0
    Curves => 1
    Squares => 2
    Stripes => 3
    """
    if data == "Circles":
        return 0
    elif data == "Curves":
        return 1
    elif data == "Squares":
        return 2
    elif data == "Stripes":
        return 3

In [815]:
def get_skip_type(data):
    """
    Return the skip type as a number.
    original => 0
    v2 => 1
    """
    if data == "original":
        return 0
    elif data == "v2":
        return 1

In [816]:
# Format data

# Find the average of the skip values
df['Skip Values'] = df['Skip Values'].map(lambda x: get_mean(x))
df['Type'] = df['Type'].map(lambda x: get_type(x))
# df.Type = df.Type.map({'Circles': 0, 'Curves': 1, 'Squares': 2, 'Stripes': 3})
df.Result = df.Result.map(lambda x: 1 if x == 0 else 0)
df['Skip Type'] = df['Skip Type'].map(lambda x: get_skip_type(x))

In [817]:
scaler = MinMaxScaler()
knn = KNeighborsClassifier(n_neighbors=8)
kmeans = KMeans(n_clusters=8)

In [818]:
# Cluster the data
kmeans.fit(df[["Modulus", "Size", "Skip Values", "Type"]])
df["Cluster"] = kmeans.predict(df[["Modulus", "Size", "Skip Values", "Type"]])
df.head()

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.

In [ ]:
# df = df.drop(columns=['Modulus', 'Size', 'Skip Values'], axis=1)
features = [["Modulus", "Size", "Skip Values", "Type", "Cluster"]]
for feature in features:
    df[feature] = scaler.fit_transform(df[feature])
df.head()
# df = df.drop(columns=["Modulus", "Size", "Skip Values"], axis=1)
# df.head()

,Type,Modulus,Size,Skip Values,Skip Type,Result,Cluster
0,0.666667,0.555556,0.010204,0.772727,0,0,1.000000
1,0.000000,0.333333,0.295918,0.318182,0,0,0.428571
2,0.000000,0.703704,0.112245,0.181818,0,0,0.857143
3,0.666667,0.481481,0.214286,0.772727,0,1,1.000000
4,1.000000,0.629630,0.459184,0.261364,0,0,0.142857


In [ ]:
# Seperate data
X = df.drop(columns=["Result"], axis=1)
y = df.Result
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
knn.fit(X_train.values, y_train.values)
y_pred = knn.predict(X_test.values)

In [ ]:
# Check accuracy, score, and roc_auc_score
print(accuracy_score(y_test.values, y_pred))
print(knn.score(X_test.values, y_test.values))
print(roc_auc_score(y_test.values, y_pred))

0.7735849056603774
0.7735849056603774
0.5943223443223442
